In [2]:
from azureml.core import Workspace, Experiment, ScriptRunConfig, Environment

# Connect to Azure ML Workspace
ws = Workspace.from_config()  # Ensure your `config.json` file is present

# Create an Azure ML experiment
experiment = Experiment(ws, "breast_pytorch_nn_experiment")

# Define an execution environment
env = Environment.from_conda_specification(name="pytorch-env", file_path="environment.yml")


If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [3]:
from azureml.core import Workspace
from azureml.core.compute import ComputeInstance, ComputeTarget
from azureml.exceptions import ComputeTargetException

# Load your Azure ML workspace
ws = Workspace.from_config()  # Assumes you have a config.json file

# Define compute instance name
compute_name = "compute-hehe"

# Set VM size (adjust as needed)
vm_size = "Standard_DS3_v2"

try:
    # Check if the compute instance already exists
    compute_instance = ComputeInstance(ws, compute_name)
    print(f"Compute instance {compute_name} already exists.")

except ComputeTargetException:
    print(f"Creating new compute instance: {compute_name}")

    compute_config = ComputeInstance.provisioning_configuration(vm_size=vm_size)

    compute_instance = ComputeInstance.create(ws, compute_name, compute_config)
    compute_instance.wait_for_completion(show_output=True)

print(f"Compute instance '{compute_name}' is ready!")


Creating new compute instance: compute-hehe

Creating...................................................
Running
Compute instance 'compute-hehe' is ready!


In [2]:
# Set up the script configuration
compute_name = "compute-hehe"
script_config = ScriptRunConfig(
    source_directory=".",  # Path to the script folder
    script="train.py",  
    compute_target=compute_name,  # Change to your Azure ML compute name
    environment=env,
    arguments=["--epochs", 20, "--lr", 0.001]
)

In [3]:
# Submit experiment
run = experiment.submit(script_config)
print("Experiment submitted! Tracking in Azure ML Studio.")
run.wait_for_completion(show_output=True)


Experiment submitted! Tracking in Azure ML Studio.
RunId: breast_pytorch_nn_experiment_1743245135_77488d99
Web View: https://ml.azure.com/runs/breast_pytorch_nn_experiment_1743245135_77488d99?wsid=/subscriptions/15d6b460-1d61-4813-914e-eb2d235bceb1/resourcegroups/Azure_ML/workspaces/Azure_ml&tid=b32f7980-cc9b-4fad-a6b9-c7f1c515d471

Streaming azureml-logs/20_image_build_log.txt

======Starting Image Build on Serverless Compute======
The run ID for the image build on serverless compute is imgbldrun_72611d7
Additional logs for the run: https://ml.azure.com/experiments/id/prepare_image/runs/imgbldrun_72611d7?wsid=/subscriptions/15d6b460-1d61-4813-914e-eb2d235bceb1/resourcegroups/Azure_ML/workspaces/Azure_ml&tid=b32f7980-cc9b-4fad-a6b9-c7f1c515d471
2025-03-29T10:49:09: Logging into Docker registry: 7dc8b21fb64844b8b5b98b3d06f4ce85.azurecr.io
2025-03-29T10:49:09: WARNING! Using --password via the CLI is insecure. Use --password-stdin.
2025-03-29T10:49:09: WARNING! Your password will be stor

{'runId': 'breast_pytorch_nn_experiment_1743245135_77488d99',
 'target': 'compute-hehe',
 'status': 'Completed',
 'startTimeUtc': '2025-03-29T10:56:05.26727Z',
 'endTimeUtc': '2025-03-29T10:57:05.643403Z',
 'services': {},
 'properties': {'_azureml.ComputeTargetType': 'amlcdsi',
  '_azureml.ClusterName': 'compute-hehe',
  'ContentSnapshotId': 'a4671c37-49fd-452e-b47e-9c9427700660',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'script': 'train.py',
  'command': '',
  'useAbsolutePath': False,
  'arguments': ['--epochs', '20', '--lr', '0.001'],
  'sourceDirectoryDataStore': None,
  'framework': 'Python',
  'communicator': 'None',
  'target': 'compute-hehe',
  'dataReferences': {},
  'data': {},
  'outputData': {},
  'datacaches': [],
  'jobName': None,
  'maxRunDurationSeconds': 2592000,
  'nodeCount': 1,
  'instanceTypes': [],
  'priority': None,
  'crede

In [4]:
from azureml.core import Workspace, Model

ws = Workspace.from_config()
model_list = Model.list(ws, name="breast_cancer_nn")

if model_list:
    print(f"Model found: {model_list[-1].name} (Version {model_list[-1].version})")
else:
    print("No registered model found!")


Model found: breast_cancer_nn (Version 5)


In [1]:
from azureml.core.compute import ComputeTarget
from azureml.exceptions import ComputeTargetException
try:
    # Retrieve the compute instance
    compute_instance = ComputeTarget(workspace=ws, name=compute_name)

    # Confirm before deleting
    user_input = input(f"Are you sure you want to delete '{compute_name}'? (yes/no): ").strip().lower()
    if user_input == "yes":
        compute_instance.delete()
        print(f"Deletion of compute instance '{compute_name}' initiated.")
    else:
        print("Deletion canceled.")

except ComputeTargetException:
    print(f"Compute instance '{compute_name}' not found in the workspace.")

NameError: name 'ws' is not defined